In [12]:
from dotenv import load_dotenv
import os
import openai 
import json 

# Carregar variáveis do arquivo .env
load_dotenv()

# Verificar se a chave foi carregada
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# Configurar o cliente OpenAI com a chave carregada
openai.api_key = api_key

In [13]:
client = openai.Client()

In [14]:
def calcular_imc(peso,altura):
    imc = peso/ (altura ** 2)
    if imc < 18.5:
        estado = "abaixo do peso"
        recomendacao = "É importante que voce consulte um médico para ajustar sua alimentacao" 
    elif 18.5 <= imc < 24.9:
        estado = "peso normal"
        recomendacao = "Continuar habitos saudaveis" 
    elif 25 <= imc < 29.9:
        estado = "sobrepeso"
        recomendacao = "Voce pode considerar uma reavaliacao de sua dieta e fazer exercicios fisicos" 
    else:
        estado = "obesidade"
        recomendacao = "Altamente recomendado buscar um médido para orientacoes" 
    return json.dumps({
        "imc":imc,
        "estado":estado,
        "recomendacao":recomendacao
    })

In [15]:
tools = [
    {
        "type":"function",
        "function":{
            "name":"calcular_imc",
            "description":"Calcular o IMC de uma pessoa e fornecer uma recomendacao",
            "parameters":{
                "type":"object",
                "properties":{
                    "peso":{
                        "type":"number",
                        "description":"Peso da pessoa em kg"
                    },
                    "altura":{
                        "type":"number",
                        "description":"Altura da pessoa em metros"
                    }
                },
                "required":["peso","altura"]
            }
        }
    }
]

In [16]:
funcao_disponivel = {
    "calcular_imc": calcular_imc
}

In [17]:
mensagens = [{"role":"user","content":"Qual é o IMC de uma pessoa que pesa 91 kg e tem 1.75 m de altura?"}]

In [18]:
resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo-0125",
    tools=tools,
    tool_choice="auto"
)

In [19]:
mensagem_respo = resposta.choices[0].message
mensagem_respo

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_p9SlADUfbALHXJNE6fTE4WfL', function=Function(arguments='{"peso":91,"altura":1.75}', name='calcular_imc'), type='function')])

In [20]:
tools_calls = mensagem_respo.tool_calls
tools_calls

[ChatCompletionMessageToolCall(id='call_p9SlADUfbALHXJNE6fTE4WfL', function=Function(arguments='{"peso":91,"altura":1.75}', name='calcular_imc'), type='function')]

In [21]:
if tools_calls:
    mensagens.append(mensagem_respo)
    for tool_call in tools_calls:
        function_name = tool_call.function.name 
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            peso = function_args.get("peso"),
            altura = function_args.get("altura"),
        )

        mensagens.append(
            {
                "tool_call_id":tool_call.id,
                "role":"tool",
                "name": function_name,
                "content":function_response
            }
        )
    
    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens
    )


In [22]:
mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='O IMC (Índice de Massa Corporal) dessa pessoa é de aproximadamente 29.71, o que indica que ela está com sobrepeso. Recomenda-se uma reavaliação da dieta e a prática de exercícios físicos.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)